## Make sure to execute from the root dir of the project. Or add the project path into sys.path:

In [1]:
# import sys
# sys.path.append("<FULL PATH TO THE DIR>/CEASystem/")

In [7]:
from evidence_scoring import AlignmentMarixFilteringScoring

import numpy as np

In [8]:
MNLI_MODEL_PATH = "roberta-large-mnli" # from huggingface or local

# Label decoder. Should contain entailment and contradiction keys with their location in logits of the model
LABEL_DECODER = {"entailment": 2, "contradiction": 0}

# If >= 0, the evidences that support <= min_alignment_limit claims will be removed. If -1, will be ignored
MIN_AL_LIMIT = -1 

# If >= 0, the evidences that support <= min_alignment_limit claims will be removed. If -1, will be ignored
MAX_AL_LIMIT = -1 

# batch size to use when running transformers model
BATCH_SIZE= 8

scorer = AlignmentMarixFilteringScoring(
        mnli_model_name_path=MNLI_MODEL_PATH, 
        model_label_decoder=LABEL_DECODER
    )

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
# Input: See the output of EvidenceAlignment.ipynb
alignment_matrix = np.array([[1., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0.]])

evidence_texts = [
    "natural factors can influence climate",
    "the current warming trend is largely attributed to human activities",
    "the burning of fossil fuels and deforestation cause climate change",
    "The financial crisis of 1929 was caused by too much cash in my pocket",
    "A balanced diet with a variety of nutrients is more beneficial for overall health",
    "Excessive consumption of chocolate, especially with added sugars, can contribute to health problems",
    "dogs can bark"
]

hypothesis_texts = [
    "Climate change is not influenced by human activities",
    "Eating chocolate every day is good for your health"
]

In [10]:
pred_dict = scorer.score_filter_matrix(
    alignment_matrix=alignment_matrix,
    evidence_texts = evidence_texts,
    hypothesis_texts=hypothesis_texts,
    min_alignment_limit=MIN_AL_LIMIT,
    max_alignment_limit=MAX_AL_LIMIT,
    batch_size=BATCH_SIZE
)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


## See README for more details about the output keys and formats

In [11]:
pred_dict.keys()

dict_keys(['scoring_matrix', 'filtered_evidences', 'hypothesis_texts', 'dropped_evidences'])

In [12]:
pred_dict["scoring_matrix"]

[[0.03561151400208473,
  -0.9924881458282471,
  -0.8473494052886963,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0, 0.0, 0.0, 0.0, 0.06036905571818352, -0.9275469183921814, 0.0]]

In [13]:
pred_dict["filtered_evidences"]

['natural factors can influence climate',
 'the current warming trend is largely attributed to human activities',
 'the burning of fossil fuels and deforestation cause climate change',
 'The financial crisis of 1929 was caused by too much cash in my pocket',
 'A balanced diet with a variety of nutrients is more beneficial for overall health',
 'Excessive consumption of chocolate, especially with added sugars, can contribute to health problems',
 'dogs can bark']

In [14]:
pred_dict["hypothesis_texts"]

['Climate change is not influenced by human activities',
 'Eating chocolate every day is good for your health']

In [15]:
pred_dict["dropped_evidences"]

[]